<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter12/Detectron2_Chapter12_CPPEnvironment.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 - Deploying into C++ environemnt

## Installing LibTorch

In [ ]:
!wget --quiet https://download.pytorch.org/libtorch/nightly/cpu/libtorch-shared-with-deps-latest.zip
!unzip -qq libtorch-shared-with-deps-latest.zip

## Make a folder for the application

In [ ]:
import os
os.makedirs('cppapp', exist_ok=True)

## Write a CMake.txt file

In [ ]:
%%writefile cppapp/CMakeLists.txt
cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(cppapp)

find_package(Torch REQUIRED)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} ${TORCH_CXX_FLAGS}")

add_executable(cppapp cppapp.cpp)
target_link_libraries(cppapp "${TORCH_LIBRARIES}")
set_property(TARGET cppapp PROPERTY CXX_STANDARD 14)

# On Windows, add the following too
if (MSVC)
  file(GLOB TORCH_DLLS "${TORCH_INSTALL_PREFIX}/lib/*.dll")
  add_custom_command(TARGET cppapp
                     POST_BUILD
                     COMMAND ${CMAKE_COMMAND} -E copy_if_different
                     ${TORCH_DLLS}
                     $<TARGET_FILE_DIR:cppapp>)
endif (MSVC)

Writing cppapp/CMakeLists.txt


In [ ]:
%%writefile cppapp/cppapp.cpp
#include <torch/script.h>
#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
  torch::jit::script::Module module = torch::jit::load(argv[1]);
  std::cout << "Loaded the module\n";
  // Create a vector of inputs.
  float data[] = {1, 2, 3, 4};
  torch::Tensor tensor = torch::from_blob(data, {4});
  std::vector<torch::jit::IValue> inputs;
  inputs.push_back(tensor);
  at::Tensor output = module.forward(inputs).toTensor();
  std::cout << "pred result" << output << '\n';
}

Overwriting cppapp/cppapp.cpp


In [ ]:
os.makedirs('cppapp/build', exist_ok=True)

In [ ]:
cd cppapp/build

/content/cppapp/build


In [ ]:
!cmake -DCMAKE_PREFIX_PATH=/content/libtorch ..
!cmake --build . --config Release

-- Configuring done
-- Generating done
-- Build files have been written to: /content/cppapp/build
Consolidate compiler generated dependencies of target cppapp
[ 50%] Building CXX object CMakeFiles/cppapp.dir/cppapp.cpp.o
[100%] Linking CXX executable cppapp
[100%] Built target cppapp


## Test the application

In [ ]:
## Load a model
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter12/scripted_model.pt

--2023-02-12 23:05:51--  https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter12/scripted_model.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter12/scripted_model.pt [following]
--2023-02-12 23:05:52--  https://raw.githubusercontent.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter12/scripted_model.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3537 (3.5K) [application/octet-stream]
Saving to: ‘scripted_model.pt.1’

scripted_model.pt.1 100%[===================

In [ ]:
## run it
!./cppapp "scripted_model.pt"

Loaded the module
pred result 0.1100
[ CPUFloatType{1} ]
